In [56]:
from pyspark import SparkContext
import operator
sc = SparkContext()
links = sc.parallelize([('A',['B','C']),('B',['A','C']),('C',['A','B','D']),('D',['C'])])
#两种转化方式对比。
links2 = sc.parallelize({'A':['B','C'],'B':['A','C'],'C':['A','B','D'],'D':['C']}.items())
ranks = links.mapValues(lambda x: 1.0)
iterMax = 10
for i in range(iterMax):
    contributions = links.join(ranks).flatMapValues(lambda x:(map(lambda i:(i,x[1]/len(x[0])),x[0]))).values()
    ranks = contributions.reduceByKey(operator.add).mapValues(lambda x:0.15+0.85*x)
print ranks.collect()
#两种计算pagerank值的方式。最后可以看到结果是一样的。
ranks = links2.mapValues(lambda x: 1.0)
def fun(x):
    dests = x[1][0]
    n = len(dests)
    rank = x[1][1]
    result = []
    for i in dests:
        result.append((i,rank*1.0/n))
    return result
for i in range(iterMax):
    contributions = links2.join(ranks).flatMap(fun)
    ranks = contributions.reduceByKey(operator.add).mapValues(lambda x:0.15+0.85*x)
print ranks.collect()

[('C', 1.4621033282930214), ('B', 0.9850243302878132), ('A', 0.9850243302878132), ('D', 0.5678480111313515)]
[('C', 1.4621033282930214), ('B', 0.9850243302878132), ('A', 0.9850243302878132), ('D', 0.5678480111313515)]
